In [1]:
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
# from torch.utils.data import Dataset, DataLoader -> datasets 라이브러리랑 충돌

c:\Users\ehddl\AppData\Local\pypoetry\Cache\virtualenvs\sns-categorizer-wO1G7-CE-py3.11\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(torch.cuda.is_available())
print(torch.cuda.device_count()) 
print(torch.cuda.get_device_name(0))  

True
1
NVIDIA GeForce RTX 3050


In [ ]:
# pip install torch==2.5.1+cu121 --index-url https://download.pytorch.org/whl/cu121
# poetry run pip install torch==2.5.1+cu121 --index-url https://download.pytorch.org/whl/cu121

# torch gpu버전을 pip install torch==2.5.1+cu121 --index-url https://download.pytorch.org/whl/cu121로 설치
# huggingface transformer 라이브러리가 pytorch 2.6 미만 버전은 보안 이슈로 강제로 차단하는데, gpu 버전의 torch는 현재 pip로 설치가 불가능. 따라서 우회하는 user_safetensor를 추가

In [3]:
os.chdir("C:/Users/ehddl/Desktop/업무/code/sns-categorizer/")

In [4]:
data = pd.read_csv("tests/final_fine_tuning_augmented_data.csv", index_col=0)
data

,text,label_id
0,휴가 돌려줘,19
1,관종들 릴스 리스타 그램 릴스 초보,21
2,날이 좋아서,19
3,행복했던 9월 고마워,19
4,협찬 동결건조야채 블록 1개로 13종의 보라 야채와 유산균 섭취 가능 보라색 안토시...,8
...,...,...
42919,"아름다운 색채의 향연, 수채화로 그려낸 풍경화.",8
42920,"예술가의 아침, 커피와 함께하는 스케치 시간.",8
42921,"디자인 스튜디오에서의 하루, 창의력 넘치는 공간.",8
42922,"도시의 밤을 담은 사진, 빛과 그림자의 조화.",8


In [6]:
# data["spelled_checked_text"] = data["spelled_checked_text"].fillna("")  # NaN → 빈 문자열
# data["spelled_checked_text"] = data.apply(
#     lambda row: row["media_cn_cleaned"] if row["spelled_checked_text"] == "" else row["spelled_checked_text"],
#     axis=1)

In [7]:
# data.drop(['media_cn_cleaned'], axis=1, inplace=True)

In [5]:
train, test = train_test_split(data, test_size= 0.2, stratify=data['label_id'], random_state=42)

In [6]:
# huggingface의 datasets.Dataset을 사용하는 방식

dataset = DatasetDict({
    'train' : Dataset.from_pandas(train),
    'test' : Dataset.from_pandas(test)
})

##### basic code & hyper parameter tuning

In [40]:
# model & toknizer loading

model_name = "BM-K/KoSimCSE-roberta" # BM-K/KoSimCSE-roberta, kykim/bert-kor-base
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# tokenizer = BertTokenizerFast.from_pretrained(model_name)

## 모델이 bert 계열이고, 속도가 중요하면 BertTkenizerFast
## 모델을 자주 비꾸거나 여러 모델 실험할 계획이면 AutoTokenizer

In [41]:
# max_length 최적화를 위해서 ids의 길이 확인
# 길이 계산 함수 정의
def get_tokenized_length(example):
    # truncation=True를 통해 max_length를 초과하는 텍스트는 잘립니다.
    # 모델에 실제로 들어갈 길이만 측정하는 것이 중요합니다.
    # padding은 길이를 측정하는 단계에서는 필요하지 않으므로 사용하지 않습니다.
    tokenized_output = tokenizer(example["text"], truncation=True)
    return {"length": len(tokenized_output["input_ids"])}

# dataset['train']의 'text' 컬럼에 대해 길이 계산
# map 함수는 새로운 컬럼('length')을 추가하여 반환합니다.
lengths_dataset = dataset["train"].map(get_tokenized_length, batched=False, remove_columns=dataset["train"].column_names)

# 'length' 컬럼에서 모든 길이를 numpy 배열로 추출
text_lengths = np.array(lengths_dataset['length'])

# 퍼센타일 값 계산
percentile_90 = np.percentile(text_lengths, 90)
percentile_95 = np.percentile(text_lengths, 95)
percentile_99 = np.percentile(text_lengths, 99)
max_length_observed = np.max(text_lengths)

print(f"훈련 데이터 문장 길이 통계 (토큰 기준):")
print(f"90th 퍼센타일: {percentile_90}")
print(f"95th 퍼센타일: {percentile_95}")
print(f"99th 퍼센타일: {percentile_99}")
print(f"가장 긴 문장 길이 (Truncation 적용 후): {max_length_observed}")

Map: 100%|██████████| 34339/34339 [00:20<00:00, 1683.33 examples/s]


훈련 데이터 문장 길이 통계 (토큰 기준):
90th 퍼센타일: 292.0
95th 퍼센타일: 360.0
99th 퍼센타일: 512.0
가장 긴 문장 길이 (Truncation 적용 후): 512


In [43]:
def tokenize_fn(ex):
    return tokenizer(ex["text"], padding="max_length", truncation=True, max_length=128) # max_length 조정 (128->512)

dataset = dataset.map(tokenize_fn, batched=True)
dataset = dataset.rename_column("label_id", "label")
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])


Map: 100%|██████████| 8585/8585 [00:00<00:00, 11196.45 examples/s]


In [44]:
print(dataset['train'])
print(dataset['test'])

Dataset({
    features: ['text', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 34339
})
Dataset({
    features: ['text', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 8585
})


In [45]:
# setting model

num_labels = data['label_id'].nunique()
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    use_safetensors=True # gpu 버전 사용 시 추가
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at BM-K/KoSimCSE-roberta and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [46]:
# setting trainer
# !pip install accelerate>=0.26.0

args = TrainingArguments(
    output_dir="finetune-BM-K", # finetune-BM-K
    eval_strategy="epoch", # evaluation_strategy -> eval_strategy
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5, # 3 -> 10
    learning_rate=2e-5,
    warmup_ratio=0.06, # 전체 학습 스템의 6% 동안 학습률을 0부터 learning_rate 까지 선형적으로 증가
    weight_decay=0.01, # 정규화 진행
    logging_dir="./logs",
    logging_steps=100,
    save_total_limit=1,
    load_best_model_at_end=True, # 학습 종료 시 검증 세트에서 가장 성능이 좋았던 모델 로드
    metric_for_best_model="accuracy",
    greater_is_better=True # accuracy는 높을수록 좋으므로 True
)

In [47]:
def compute_metrics(p):
    preds = p.predictions.argmax(axis=-1)
    labels = p.label_ids
    return {"accuracy": accuracy_score(labels, preds)}

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    compute_metrics=compute_metrics
)

In [48]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.097200,1.059403,0.707863
2,0.865400,0.971543,0.725684
3,0.711000,1.031504,0.723588
4,0.488200,1.082983,0.719161
5,0.397700,1.134532,0.721840


TrainOutput(global_step=10735, training_loss=0.8034730520368344, metrics={'train_runtime': 7938.8275, 'train_samples_per_second': 21.627, 'train_steps_per_second': 1.352, 'total_flos': 1.129614680315136e+16, 'train_loss': 0.8034730520368344, 'epoch': 5.0})

In [36]:
trainer.save_model("finetune-bert-kor")
tokenizer.save_pretrained("finetune-bert-kor") 

('finetune-bert-kor\\tokenizer_config.json',
 'finetune-bert-kor\\special_tokens_map.json',
 'finetune-bert-kor\\vocab.txt',
 'finetune-bert-kor\\added_tokens.json',
 'finetune-bert-kor\\tokenizer.json')

In [50]:
trainer.save_model("finetune-BM-K")
tokenizer.save_pretrained("finetune-BM-K") 

('finetune-BM-K\\tokenizer_config.json',
 'finetune-BM-K\\special_tokens_map.json',
 'finetune-BM-K\\vocab.txt',
 'finetune-BM-K\\added_tokens.json',
 'finetune-BM-K\\tokenizer.json')

##### ensemble - voting

In [7]:
num_labels = data['label_id'].nunique()

In [8]:
model_name_1 = "kykim/bert-kor-base"
model_path_1 = "finetune-bert-kor"

model1 = AutoModelForSequenceClassification.from_pretrained(model_path_1, num_labels=num_labels)
model1.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(42000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [9]:
model_name_2 = "BM-K/KoSimCSE-roberta"
model_path_2 = "finetune-BM-K"

model2 = AutoModelForSequenceClassification.from_pretrained(model_path_2, num_labels=num_labels)
model2.eval()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [10]:
# 불필요한 __index_level_0__ 컬럼 제거 (DataFrame.to_dict() 시 생길 수 있음)
if "__index_level_0__" in dataset["train"].column_names:
    dataset["train"] = dataset["train"].remove_columns(["__index_level_0__"])
if "__index_level_0__" in dataset["test"].column_names:
    dataset["test"] = dataset["test"].remove_columns(["__index_level_0__"])

In [11]:
# --- 0.4. 각 모델에 맞는 토크나이저 로드 및 텍스트 토큰화 ---

# kykim 모델용 토크나이저 및 데이터셋
tokenizer_kykim = AutoTokenizer.from_pretrained("kykim/bert-kor-base")
def tokenize_fn_kykim(examples):
    return tokenizer_kykim(examples["text"], padding="max_length", truncation=True, max_length=512)

dataset_kykim_tokenized = dataset.map(tokenize_fn_kykim, batched=True)
dataset_kykim_tokenized = dataset_kykim_tokenized.rename_column("label_id", "label")
dataset_kykim_tokenized.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

# KoSimCSE 모델용 토크나이저 및 데이터셋
# BM-K/KoSimCSE-roberta 모델은 'klue/roberta-base' 토크나이저를 기반으로 할 가능성이 높습니다.
# 정확한 토크나이저 이름은 'BM-K/KoSimCSE-roberta' 모델 페이지에서 확인하는 것이 좋습니다.
tokenizer_kosimcse = AutoTokenizer.from_pretrained("BM-K/KoSimCSE-roberta") # <-- 해당 모델의 토크나이저!
def tokenize_fn_kosimcse(examples):
    # KoSimCSE 토크나이저는 token_type_ids를 생성하지 않을 수 있습니다.
    # 하지만 RoBERTa 모델이 내부적으로 이 인자를 기대할 수 있으니
    # 필요하다면 아래와 같이 0으로 채워넣는 로직을 추가해봅니다. (시험적)
    tokenized_output = tokenizer_kosimcse(examples["text"], padding="max_length", truncation=True, max_length=512)
    if "token_type_ids" not in tokenized_output:
        tokenized_output["token_type_ids"] = [[0] * len(ids) for ids in tokenized_output["input_ids"]]
    return tokenized_output

dataset_kosimcse_tokenized = dataset.map(tokenize_fn_kosimcse, batched=True)
dataset_kosimcse_tokenized = dataset_kosimcse_tokenized.rename_column("label_id", "label")
# 중요한 점: kosimcse 모델이 token_type_ids를 기대한다면 set_format에도 추가
dataset_kosimcse_tokenized.set_format(type="torch", columns=["input_ids", "attention_mask", "token_type_ids", "label"])
# 만약 token_type_ids를 넣었는데도 에러나면 다시 빼보고 테스트

# --- 2. Trainer를 사용하여 각 모델의 예측 로짓(Logits) 얻기 ---
# Trainer를 사용하면 편리하게 예측을 수행할 수 있습니다.
# 예측을 위한 Trainer는 학습 인자가 필요 없으므로 간소화합니다.
# eval_dataset 대신 test_dataset을 사용한다고 가정

# 더미 TrainingArguments (예측용이므로 대부분의 인자는 중요하지 않음)
prediction_args = TrainingArguments(
    output_dir="./voting_prediction_output", # 임시 출력 디렉토리
    per_device_eval_batch_size=16, # 평가 배치 크기
    do_train=False,
    do_predict=True,
    report_to="none", # W&B 등 로깅 비활성화
    remove_unused_columns=False
)

# Model 1 Prediction
trainer1 = Trainer(model=model1, args=prediction_args)
predictions1 = trainer1.predict(dataset_kykim_tokenized["test"])
logits1 = predictions1.predictions # numpy array 형태의 로짓

# Model 2 Prediction
trainer2 = Trainer(model=model2, args=prediction_args)
predictions2 = trainer2.predict(dataset_kosimcse_tokenized["test"])
logits2 = predictions2.predictions

# --- 3. 로짓(Logits) 또는 확률(Probabilities) 평균내기 ---
# 소프트 보팅은 로짓 또는 softmax 확률을 평균낼 수 있습니다.
# 일반적으로 로짓을 평균내는 것이 더 안정적이라고 알려져 있습니다.
# (softmax는 비선형 변환이므로 평균 내기 전에 로짓 상태에서 평균 내는 것이 좋습니다.)

# 로짓 평균
ensemble_logits = (logits1 + logits2) / 2 # 모델이 3개라면 (logits1 + logits2 + logits3) / 3

# 평균 로짓에서 최종 예측 클래스 선택
ensemble_predictions = np.argmax(ensemble_logits, axis=-1)

# 실제 라벨 가져오기 (predictions1 객체에서 가져오는 것이 편리)
true_labels = predictions1.label_ids

# --- 4. 앙상블 성능 평가 ---
ensemble_accuracy = accuracy_score(true_labels, ensemble_predictions)

print(f"모델 1 ({model_name_1})의 테스트 정확도: {accuracy_score(true_labels, np.argmax(logits1, axis=-1)):.4f}")
print(f"모델 2 ({model_name_2})의 테스트 정확도: {accuracy_score(true_labels, np.argmax(logits2, axis=-1)):.4f}")
print(f"앙상블 모델의 테스트 정확도 (소프트 보팅): {ensemble_accuracy:.4f}")

Map: 100%|██████████| 8585/8585 [00:01<00:00, 4821.62 examples/s]


모델 1 (kykim/bert-kor-base)의 테스트 정확도: 0.7600
모델 2 (BM-K/KoSimCSE-roberta)의 테스트 정확도: 0.7437
앙상블 모델의 테스트 정확도 (소프트 보팅): 0.7609


##### Inference

In [19]:
# model_name_1 = "kykim/bert-kor-base"
# model_path_1 = "finetune-bert-kor"

# model = AutoModelForSequenceClassification.from_pretrained(model_path_1, num_labels=num_labels)
# model_tokenizer = AutoTokenizer.from_pretrained("kykim/bert-kor-base")
# model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(42000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [30]:
model_name_2 = "BM-K/KoSimCSE-roberta"
model_path_2 = "finetune-BM-K"

num_labels = data['label_id'].nunique()
model = AutoModelForSequenceClassification.from_pretrained(model_path_2, num_labels=num_labels)
model_tokenizer = AutoTokenizer.from_pretrained("BM-K/KoSimCSE-roberta")
model.eval()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [ ]:
data_for_label = data.copy()
data_for_label.drop(['label_id'], axis=1, inplace=True) 
predict_dataset = Dataset.from_pandas(data_for_label)

In [32]:
def tokenize_fn_for_prediction(ex):
    # label 칼럼이 없으므로 text 컬럼만 토큰화 진행
    return model_tokenizer(ex["text"], padding="max_length", truncation=True, max_length=128) # max_length 조정 (128->512)

predict_dataset = predict_dataset.map(tokenize_fn_for_prediction, batched=True)
predict_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"]) # 라벨은 없으므로 제외


Map: 100%|██████████| 42924/42924 [00:04<00:00, 10517.43 examples/s]


In [33]:
prediction_args = TrainingArguments(
    output_dir="./prediction_output", # 임시 출력 디렉토리
    per_device_eval_batch_size=16, # 평가 배치 크기
    do_train=False, # 학습은 안 함
    do_predict=True, # 예측만 수행
    report_to="none", # W&B 등 로깅 비활성화
    disable_tqdm=False, # 진행률 바 표시 (기본값 True)
)

# Trainer 인스턴스 생성
# 이전에 학습에 사용한 compute_metrics 함수는 필요 X (라벨이 없으므로)
trainer = Trainer(model=model, args=prediction_args)

# 예측 수행
predictions_output = trainer.predict(predict_dataset)

# predictions_output 객체에서 로짓 추출
logits = predictions_output.predictions
print(f"\n예측된 로짓 형태: {logits.shape}")

# 로짓을 확률로 변환 (필요한 경우)
probabilities = torch.softmax(torch.tensor(logits), dim=-1).numpy()

# 가장 높은 확률을 가진 라벨의 인덱스 추출
predicted_class_indices = np.argmax(logits, axis=-1)

# 라벨 인덱스를 실제 라벨 이름으로 매핑
category_labels = [f'{i}' for i in range(num_labels)]
predicted_label_names = [category_labels[idx] for idx in predicted_class_indices]


예측된 로짓 형태: (42924, 26)


In [ ]:
data_for_label['predicted_label'] = predicted_label_names
data_for_label['predicted_label'] = data_for_label['predicted_label'].astype(int)
predict1_accuracy = accuracy_score(data_for_label['label_id'], data_for_label['predicted_label'])
print(predict1_accuracy)

In [36]:
data_for_label

,text,label_id,predicted_label
0,휴가 돌려줘,19,19
1,관종들 릴스 리스타 그램 릴스 초보,21,14
2,날이 좋아서,19,19
3,행복했던 9월 고마워,19,19
4,협찬 동결건조야채 블록 1개로 13종의 보라 야채와 유산균 섭취 가능 보라색 안토시...,8,8
...,...,...,...
42919,"아름다운 색채의 향연, 수채화로 그려낸 풍경화.",8,8
42920,"예술가의 아침, 커피와 함께하는 스케치 시간.",8,8
42921,"디자인 스튜디오에서의 하루, 창의력 넘치는 공간.",8,8
42922,"도시의 밤을 담은 사진, 빛과 그림자의 조화.",8,8


In [ ]:
# data_for_label.to_csv("tests/kykim_label_data.csv")

In [38]:
data_for_label.to_csv("tests/bm-k_label_data.csv")

In [79]:
for i, label_name in enumerate(category_labels):
    data_for_label[f'prob_{label_name}'] = probabilities[:, i]

In [19]:
data_for_label.head()

,text
0,휴가 돌려줘
1,관종들 릴스 리스타 그램 릴스 초보
2,날이 좋아서
3,행복했던 9월 고마워
4,협찬 동결건조야채 블록 1개로 13종의 보라 야채와 유산균 섭취 가능 보라색 안토시...


ensemble model prediction

In [25]:
data_for_label = data.copy()
data_for_label.drop(['label_id'], axis=1, inplace=True) 
predict_dataset = Dataset.from_pandas(data_for_label)

In [26]:
model_name_1 = "kykim/bert-kor-base"
model_path_1 = "finetune-bert-kor"

model1 = AutoModelForSequenceClassification.from_pretrained(model_path_1, num_labels=num_labels)
model1.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(42000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [27]:
model_name_2 = "BM-K/KoSimCSE-roberta"
model_path_2 = "finetune-BM-K"

num_labels = data['label_id'].nunique()
model2 = AutoModelForSequenceClassification.from_pretrained(model_path_2, num_labels=num_labels)
model2.eval()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [28]:
# --- 0.4. 각 모델에 맞는 토크나이저 로드 및 텍스트 토큰화 ---

# kykim 모델용 토크나이저 및 데이터셋
tokenizer_kykim = AutoTokenizer.from_pretrained("kykim/bert-kor-base")
def tokenize_fn_kykim_for_predict(examples):
    return tokenizer_kykim(examples["text"], padding="max_length", truncation=True, max_length=128)

predict_dataset_kykim_tokenized = predict_dataset.map(tokenize_fn_kykim_for_predict, batched=True)
predict_dataset_kykim_tokenized.set_format(type="torch", columns=["input_ids", "attention_mask"])

# KoSimCSE 모델용 토크나이저 및 데이터셋
tokenizer_kosimcse = AutoTokenizer.from_pretrained("BM-K/KoSimCSE-roberta")
def tokenize_fn_kosimcse_for_predict(examples):
    tokenized_output = tokenizer_kosimcse(examples["text"], padding="max_length", truncation=True, max_length=128)
    if "token_type_ids" not in tokenized_output:
        tokenized_output["token_type_ids"] = [[0] * len(ids) for ids in tokenized_output["input_ids"]]
    return tokenized_output

predict_dataset_kosimcse_tokenized = predict_dataset.map(tokenize_fn_kosimcse_for_predict, batched=True)
predict_dataset_kosimcse_tokenized.set_format(type="torch", columns=["input_ids", "attention_mask", "token_type_ids"])

# --- 2. Trainer를 사용하여 각 모델의 예측 로짓(Logits) 얻기 ---
# 더미 TrainingArguments (예측용이므로 대부분의 인자는 중요하지 않음)
prediction_args = TrainingArguments(
    output_dir="./voting_prediction_output", # 임시 출력 디렉토리
    per_device_eval_batch_size=16, # 평가 배치 크기
    do_train=False,
    do_predict=True,
    report_to="none", # W&B 등 로깅 비활성화
    remove_unused_columns=False
)

# Model 1 Prediction
trainer1 = Trainer(model=model1, args=prediction_args)
predictions1 = trainer1.predict(predict_dataset_kykim_tokenized)
logits1 = predictions1.predictions # numpy array 형태의 로짓

# Model 2 Prediction
trainer2 = Trainer(model=model2, args=prediction_args)
predictions2 = trainer2.predict(predict_dataset_kosimcse_tokenized)
logits2 = predictions2.predictions


Map: 100%|██████████| 42924/42924 [00:03<00:00, 11251.78 examples/s]


In [ ]:
# --- 3. 로짓(Logits) 또는 확률(Probabilities) 평균내기 ---
# 소프트 보팅은 로짓 또는 softmax 확률을 평균낼 수 있음 (softmax는 비선형 변환이므로 평균 내기 전에 로짓 상태에서 평균 내는 것이 좋음)
# 일반적으로 로짓을 평균내는 것이 더 안정적

# 로짓 평균
ensemble_logits = (logits1 + logits2) / 2 
print(f"\n예측된 로짓 형태: {ensemble_logits.shape}")

# 로짓을 확률로 변환 (필요한 경우)
probabilities = torch.softmax(torch.tensor(ensemble_logits), dim=-1).numpy()

# 평균 로짓에서 최종 예측 클래스 선택
ensemble_predictions = np.argmax(ensemble_logits, axis=-1)

# 라벨 인덱스를 실제 라벨 이름으로 매핑
category_labels = [f'{i}' for i in range(num_labels)]
predicted_label_names = [category_labels[idx] for idx in ensemble_predictions]

In [35]:
data_for_label['predicted_label'] = predicted_label_names
data_for_label['predicted_label'] = data_for_label['predicted_label'].astype(int)
predict_accuracy = accuracy_score(data['label_id'], data_for_label['predicted_label'])
print(predict_accuracy)

0.8230127667505358


In [37]:
data_for_label.to_csv("tests/ensemble_label_data.csv")

##### Multi-column classification tuning

##### multi-label classification tuning

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

In [ ]:
# model & toknizer loading

model_name = "kykim/bert-kor-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
# tokenizer = BertTokenizerFast.from_pretrained(model_name)

## 모델이 bert 계열이고, 속도가 중요하면 BertTkenizerFast
## 모델을 자주 비꾸거나 여러 모델 실험할 계획이면 AutoTokenizer

In [ ]:
# huggingface의 datasets.Dataset을 사용하는 방식

dataset = DatasetDict({
    'train' : Dataset.from_pandas(train),
    'test' : Dataset.from_pandas(test)
})


In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label_id', '__index_level_0__'],
        num_rows: 34339
    })
    test: Dataset({
        features: ['text', 'label_id', '__index_level_0__'],
        num_rows: 8585
    })
})

In [ ]:
# max_length 최적화를 위해서 ids의 길이 확인
# 길이 계산 함수 정의
def get_tokenized_length(example):
    # truncation=True를 통해 max_length를 초과하는 텍스트는 잘립니다.
    # 모델에 실제로 들어갈 길이만 측정하는 것이 중요합니다.
    # padding은 길이를 측정하는 단계에서는 필요하지 않으므로 사용하지 않습니다.
    tokenized_output = tokenizer(example["text"], truncation=True)
    return {"length": len(tokenized_output["input_ids"])}

# dataset['train']의 'text' 컬럼에 대해 길이 계산
# map 함수는 새로운 컬럼('length')을 추가하여 반환합니다.
lengths_dataset = dataset["train"].map(get_tokenized_length, batched=False, remove_columns=dataset["train"].column_names)

# 'length' 컬럼에서 모든 길이를 numpy 배열로 추출
text_lengths = np.array(lengths_dataset['length'])

# 퍼센타일 값 계산
percentile_90 = np.percentile(text_lengths, 90)
percentile_95 = np.percentile(text_lengths, 95)
percentile_99 = np.percentile(text_lengths, 99)
max_length_observed = np.max(text_lengths)

print(f"훈련 데이터 문장 길이 통계 (토큰 기준):")
print(f"90th 퍼센타일: {percentile_90}")
print(f"95th 퍼센타일: {percentile_95}")
print(f"99th 퍼센타일: {percentile_99}")
print(f"가장 긴 문장 길이 (Truncation 적용 후): {max_length_observed}")

Map: 100%|██████████| 34339/34339 [00:08<00:00, 4135.53 examples/s]


훈련 데이터 문장 길이 통계 (토큰 기준):
90th 퍼센타일: 227.0
95th 퍼센타일: 282.0
99th 퍼센타일: 425.0
가장 긴 문장 길이 (Truncation 적용 후): 512


In [ ]:
lengths_dataset

Dataset({
    features: ['length'],
    num_rows: 34339
})

In [ ]:
def tokenize_fn(ex):
    return tokenizer(ex["text"], padding="max_length", truncation=True, max_length=512) # max_length 조정

dataset = dataset.map(tokenize_fn, batched=True)
# dataset = dataset.rename_column("label_id", "label")
# dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

Map: 100%|██████████| 8585/8585 [00:01<00:00, 5272.43 examples/s]


In [ ]:
# "하나의 텍스트가 여러개의 라벨을 가질 수 있다"는 전제 하에,
# dataset['label_id'] 컬럼이 원래부터 각 샘플에 대해 [라벨_ID_1, 라벨_ID_2, ...] 와 같은 리스트 형태여야 합니다.
# 만약 그렇지 않다면, 원본 데이터 로드 시점부터 멀티라벨 형태가 되도록 전처리해야 합니다.

# 예시: dataset['train']에 있는 모든 'label_id' 리스트를 평탄화하여 고유 라벨 추출 (멀티라벨이 이미 리스트 형태로 주어졌다고 가정)
all_labels = set()
for split in dataset.keys(): # train, test 등 모든 스플릿
    for example in dataset[split]:
        # 'label_id'가 단일 정수형이라도 여기에 추가합니다.
        # 실제 MultiLabelBinarizer는 라벨의 집합을 필요로 합니다.
        if isinstance(example['label_id'], list):
            all_labels.update(example['label_id'])
        else: # 단일 정수형 라벨인 경우
            all_labels.add(example['label_id'])

# 정렬하여 MultiLabelBinarizer에 전달 (순서 일관성 유지)
unique_labels = sorted(list(all_labels))
num_total_labels = len(unique_labels) # 27개가 맞는지 확인

mlb = MultiLabelBinarizer(classes=unique_labels)

# 라벨을 멀티라벨 이진 벡터로 변환
def convert_to_multilabel_format(example):
    # example['label_id']가 [0, 1, 3]과 같이 여러 라벨 ID를 포함하는 리스트라고 가정합니다.
    # 만약 dataset['label_id']가 여전히 단일 정수(멀티클래스)라면, 이 부분의 로직을 변경해야 합니다.
    # (예: 단일 정수를 가진 리스트로 만들고 binarize)

    # 텍스트에 할당된 라벨 ID들을 MLB를 통해 이진 벡터로 변환
    # input for mlb.fit_transform should be a list of lists (or a list of sets)
    # 각 샘플의 'label_id'가 이미 리스트 형태여야 합니다.
    # 만약 'label_id'가 단일 정수라면, `[example['label_id']]` 와 같이 리스트로 감싸줘야 합니다.
    
    # 예시: example['label_id']가 [1, 5, 10] 또는 5 와 같이 올 수 있다고 가정
    labels_to_binarize = example['label_id']
    if not isinstance(labels_to_binarize, list):
        labels_to_binarize = [labels_to_binarize] # 단일 라벨도 리스트로 감싸서 처리

    multilabel_vector = mlb.fit_transform([labels_to_binarize])[0]
    
    # PyTorch 모델에 전달하기 위해 FloatTensor로 변환
    return {"label": torch.tensor(multilabel_vector, dtype=torch.float)}

# 데이터셋의 'label_id' 컬럼을 멀티라벨 이진 벡터로 변환하고 'label' 컬럼으로 대체
dataset = dataset.map(convert_to_multilabel_format, batched=False)

# 4. 데이터셋 포맷 설정 (여기서 한 번만)
# 모든 필요한 컬럼이 포함되었는지 확인하고 'label' 컬럼의 타입을 torch.float으로 설정
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

Map:   0%|          | 0/34339 [00:00<?, ? examples/s]C:\Users\ehddl\AppData\Local\Temp\ipykernel_24404\843318086.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {"label": torch.tensor(example["label"], dtype=torch.float)}
Map: 100%|██████████| 8585/8585 [00:02<00:00, 2906.36 examples/s]


In [ ]:
# 모델 초기화 시 num_labels 설정
# num_total_labels는 MultiLabelBinarizer에 사용된 고유 라벨의 총 개수입니다.
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_total_labels, # 27개여야 함
    use_safetensors=True
)

# (Trainer 설정 및 학습은 이전과 동일)
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

In [ ]:
# setting trainer
# !pip install accelerate>=0.26.0

args = TrainingArguments(
    output_dir="finetune-bert-kor",
    eval_strategy="epoch", # evaluation_strategy -> eval_strategy
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    learning_rate=2e-5,
    logging_dir="./logs",
    logging_steps=100,
    save_total_limit=1
)

In [ ]:
def compute_metrics(p):
    preds = p.predictions.argmax(axis=-1)
    labels = p.label_ids
    return {"accuracy": accuracy_score(labels, preds)}

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.975700,0.933468,0.736517
2,0.747100,0.900717,0.746418
3,0.622600,0.917289,0.751427


TrainOutput(global_step=6441, training_loss=0.8551019856026965, metrics={'train_runtime': 3027.2064, 'train_samples_per_second': 34.03, 'train_steps_per_second': 2.128, 'total_flos': 6777688081890816.0, 'train_loss': 0.8551019856026965, 'epoch': 3.0})

Inference

In [81]:
# 모델과 토크나이저 로드
model_path = "finetune-bert-kor/"  # 저장된 모델 디렉토리
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# 모델을 평가 모드로 전환
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(42000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

##### Using Pytorch (pre-trained model)

In [ ]:
# # huggingface가 아니라 pytorch library 사용 시 방법
# from torch.utils.data import Dataset, DataLoader

# class TokenDataset(Dataset):
  
#     def __init__(self, dataframe, model_name):
#         # sentence, label 컬럼으로 구성된 데이터프레임 전달
#         self.data = dataframe        
#         # Huggingface 토크나이저 생성
#         # self.tokenizer = BertTokenizerFast.from_pretrained(tokenizer_pretrained)
#         self.tokenizer = AutoTokenizer.from_pretrained(model_name)
  
#     def __len__(self):
#         return len(self.data)
  
#     def __getitem__(self, idx):
#         sentence = self.data.iloc[idx]['document']
#         label = self.data.iloc[idx]['label']

#         # 토큰화 처리
#         tokens = self.tokenizer(
#             sentence,                
#             return_tensors='pt',     
#             truncation=True,        
#             padding='max_length',    
#             add_special_tokens=True  
#         )

#         input_ids = tokens['input_ids'].squeeze(0)           # 2D -> 1D
#         attention_mask = tokens['attention_mask'].squeeze(0) # 2D -> 1D
#         token_type_ids = torch.zeros_like(attention_mask)

#         return {
#             'input_ids': input_ids,
#             'attention_mask': attention_mask, 
#             'token_type_ids': token_type_ids,}, torch.tensor(label)

In [ ]:
# # train, test 데이터셋 생성
# train_data = TokenDataset(train, model_name)
# test_data = TokenDataset(test, model_name)

# train_loader = DataLoader(train_data, batch_size=8, shuffle=True, num_workers=8)
# test_loader = DataLoader(test_data, batch_size=8, shuffle=True, num_workers=8)

In [ ]:
# # 1개의 batch 꺼내기
# inputs, labels = next(iter(train_loader))

# # 데이터셋을 device 설정
# inputs = {k: v.to(device) for k, v in inputs.items()}
# labels.to(device)

In [ ]:
# # pad_token 없을 경우 대비
# if tokenizer.pad_token is None:
#     tokenizer.add_special_tokens({'pad_token': '[PAD]'})
#     model.resize_token_embeddings(len(tokenizer))

# def predict_label(text_list):
#     inputs = tokenizer(text_list, return_tensors="pt", padding=True, truncation=True, max_length=512)
#     with torch.no_grad():
#         outputs = model(**inputs)
#         preds = torch.argmax(outputs.logits, dim=1)
#     return preds.tolist()

# # def predict_label(text_list):
# #     inputs = tokenizer(text_list, return_tensors="pt", truncation=True, padding=True).to(model.device)
# #     with torch.no_grad():
# #         outputs = model(**inputs)
# #     probs = softmax(outputs.logits, dim=1)
# #     pred_idx = torch.argmax(probs, dim=1).item()
# #     return label_list[pred_idx]

In [ ]:
# from torch.utils.data import DataLoader

# texts = data['text'].tolist()
# batch_size = 16  # 상황에 따라 조절

# all_preds = []
# for i in tqdm(range(0, len(texts), batch_size), desc="Predicting", ncols=100):
#     batch_texts = texts[i:i+batch_size]
#     inputs = tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True, max_length=512)
#     inputs = {k: v.to(device) for k, v in inputs.items()}
#     with torch.no_grad():
#         outputs = model(**inputs)
#         preds = torch.argmax(outputs.logits, dim=1)
    
#     all_preds.extend(preds.cpu().tolist())

# # 결과를 데이터프레임에 추가
# data['bert_label'] = all_preds